# Analyses of combined states (from different models)

In [7]:

""" 
IMPORTS
"""
import os
import autograd.numpy as np
import pickle
from sklearn.preprocessing import StandardScaler, MinMaxScaler, Normalizer
import seaborn as sns
from collections import defaultdict
import pandas as pd

from one.api import ONE
from pprint import pprint
import numpy as np
import matplotlib.pyplot as plt

# Get my functions
functions_path =  '/home/ines/repositories/representation_learning_variability/Models/Sub-trial//2_fit_models/'
functions_path = '/Users/ineslaranjeira/Documents/Repositories/representation_learning_variability//Models/Sub-trial//2_fit_models/'
os.chdir(functions_path)
from preprocessing_functions import idxs_from_files, prepro_design_matrix, concatenate_sessions, fix_discontinuities
from postprocessing_functions import best_lag_kappa
# from fitting_functions import cross_validate_armodel, compute_inputs
# from plotting_functions import plot_transition_mat, plot_states_aligned, align_bin_design_matrix, states_per_trial_phase, plot_states_aligned_trial, traces_over_sates, traces_over_few_sates

one = ONE()

## Parameters

In [8]:
bin_size = 0.1

# Plotting params
multiplier = 1/bin_size

event_type_list = ['goCueTrigger_times']  # , 'feedback_times', 'firstMovement_times'
event_type_name = ['Go cue']  # , 'Feedback time', 'First movement onset'


## Data path

In [9]:
data_path =  '/home/ines/repositories/representation_learning_variability/DATA/Sub-trial/Design matrix/' + 'v4_5Jul2024/' + str(bin_size) + '/'
data_path =  '/Users/ineslaranjeira/Documents/Repositories/representation_learning_variability/DATA/Sub-trial/Design matrix/' + 'v4_5Jul2024/' + str(bin_size) + '/'

os.chdir(data_path)
design_matrices = os.listdir(data_path)

var_sets = [['avg_wheel_vel'], ['nose_X', 'nose_Y'], ['Lick count'], 
                    ['left_X', 'left_Y', 'right_X', 'right_Y'], ['whisker_me']]
var_sets = [['avg_wheel_vel'], ['nose_X', 'nose_Y'], 
                    ['left_X', 'left_Y', 'right_X', 'right_Y'], ['whisker_me']]
    
# # Load frame rate
# fr_path =  '/home/ines/repositories/representation_learning_variability/DATA/Sub-trial/Design matrix/frame_rate/'
fr_path =  '/Users/ineslaranjeira/Documents/Repositories/representation_learning_variability/DATA/Sub-trial/Design matrix/frame_rate/'

os.chdir(fr_path)
frame_rate = pickle.load(open(fr_path + "frame_rate", "rb"))


## Get mice list

In [10]:
idxs, mouse_names = idxs_from_files(one, design_matrices, frame_rate, data_path, bin_size)

## Parameters

In [11]:
threshold = 0.05
bin_size = 0.1

## Plot combined states per animal

In [13]:
var_names = ['avg_wheel_vel', 'nose_X', 'nose_Y',
              'left_X', 'left_Y', 'right_X', 'right_Y', 'whisker_me']

# Get data for all mice for the model of interest
os.chdir(data_path)
matrix_all, matrix_all_unnorm, session_all = prepro_design_matrix(one, idxs, mouse_names, 
                                                                    bin_size, var_names, data_path, first_90=True)
collapsed_matrices, collapsed_unnorm, collapsed_trials = concatenate_sessions(mouse_names, matrix_all, 
                                                                                matrix_all_unnorm, session_all)

In [16]:

# Loop through animals
for m, mat in enumerate(idxs[0:1]):
    if len(mat) > 35: 
        
        mouse_name = mat[37:]
        
        # Get design_matrix
        design_matrix = collapsed_matrices[mouse_name]
        if len(np.shape(design_matrix)) > 2:
            design_matrix = design_matrix[0]
        
        # Get states per variable and concatenate
        results_path =  '/home/ines/repositories/representation_learning_variability/DATA/Sub-trial/Results/'
        results_path =  '/Users/ineslaranjeira/Documents/Repositories/representation_learning_variability/DATA/Sub-trial/Results/'
        os.chdir(results_path)

        combined_states = []
        for s, set in enumerate(var_sets):
            
            most_likely_states = pickle.load(open("most_likely_states" + set[0] + '_' + mouse_name, "rb"))
            # Save
            if len(combined_states) > 0:
                combined_states = most_likely_states
            else:
                combined_states = np.vstack((combined_states, most_likely_states))




        

RuntimeError: This version of jaxlib was built using AVX instructions, which your CPU and/or operating system do not support. You may be able work around this issue by building jaxlib from source.

In [ ]:
        
        unique, counts = np.unique(most_likely_states, return_counts=True)
        threshold_count = threshold * len(most_likely_states)
        excluded_bins = 0
        remaining_states = list(counts.copy())
        for state in unique:
            size_smallest_state = np.nanmin(remaining_states)
            if size_smallest_state + excluded_bins < threshold_count:
                remaining_states[np.where(counts==size_smallest_state)[0][0]] = np.nan
                excluded_bins += size_smallest_state
        
        # Remove states below threshold
        new_states = np.array(most_likely_states).astype(float)
        exclude_states = np.where(np.isnan(np.array(remaining_states)))[0].astype(float)
        # Create a boolean mask to identify values to replace
        mask = np.isin(new_states, exclude_states)
        # Replace values in main_array with np.nan using the boolean mask
        new_states[mask] = np.nan

        new_states = most_likely_states
        
        " Plots "
        # Prepare data
        design_matrix_heading = pd.DataFrame(columns=var_names)
        design_matrix_heading[var_names] = design_matrix[0:len(most_likely_states)]  # TODO: need to understand why number is the same and whether it depends on lag
        bins = unnorm_mat[:(num_timesteps // num_train_batches) * num_train_batches]['Bin']
        design_matrix_heading['Bin'] = bins
        
        # Fix discontinuities
        cont_trials, cont_design_matrix_heading = fix_discontinuities(session_trials, design_matrix_heading)
        
        # Align bins
        init = -10
        end = 15
        empirical_data = align_bin_design_matrix(init, end, event_type_list, cont_trials, cont_design_matrix_heading, new_states, multiplier)

    
        """ Stack trial states """
        trial_init = 30  # First trial to plot
        trials_to_plot = 20  # How many trials to plot (if very different will compromise readibility)
        plot_states_aligned_trial(trial_init, empirical_data, cont_trials, bin_size, trials_to_plot, num_states)

        # " Plot states stacked and aligned per trial type "
        # plot_states_aligned(init, end, empirical_data, event_type_name, bin_size)
        
        # """ Plot states per trial epoch """
        # colors = sns.color_palette("viridis", num_states)
        # states_trial = states_per_trial_phase(empirical_data, session_trials)
        # plt.figure(figsize=[7, 5])
        # sns.histplot(x='label', hue=0, data=states_trial, stat = 'percent',
        #                     multiple="stack", legend=True, palette='viridis', alpha=0.3)  # stat='frequency',
        # plt.xlabel('Trial phase')
        # plt.legend(loc='upper left', bbox_to_anchor=(1, -0.5))
        # plt.xticks(rotation=45)
        # plt.tight_layout()
        # plt.title(mouse_name)
        # plt.show()
    
        """ Plot raw traces over states """
        init = 2500 # First bin
        inter = 200
        # traces_over_few_sates(init, empirical_data, session_trials, var_names)
        traces_over_few_sates(init, inter, empirical_data, cont_trials, var_names)


In [ ]:
# Get results from grid search
results_path =  '/home/ines/repositories/representation_learning_variability/DATA/Sub-trial/Results/'
os.chdir(results_path)
most_likely_states = pickle.load(open("most_likely_states" + var_names[0] + '_' + mouse_name, "rb"))
